In [ ]:
import json, requests
from IPython.display import display # import module to print an image
import random
from PIL import Image
from io import BytesIO
import os

client_id = 'xxx' # client ID for your mappilary application\n",

output_image_path = '/notebooks/mappilary scraped myself/images/'
output_json_path = '/notebooks/mappilary scraped myself/jsons/'


if not os.path.exists(output_image_path):
    os.makedirs(output_image_path)

if not os.path.exists(output_json_path):
    os.makedirs(output_json_path)
    
    
while True: 
    try:        
        ## Sample an area inside a specific area. 
        ## These are the coordinates for the Munich area. 
        ## For more fun: set them to your local area!
        long = random.uniform(11.32, 11.78)
        lat = random.uniform(48.01, 48.26)

        distance = 1500 # the maximum distance in meters that our image should be from the point it looks toward

        coordinates = "{0:2f},{1:2f}".format(long, lat)
        print(coordinates)
        
        # API call URL to get a selection of images close to that position
        url = ('https://a.mapillary.com/v3/images?client_id={}&per_page=200&lookat={}&closeto={}&radius={}').format(client_id,coordinates,coordinates,distance) 

        # request a JSON showing the point location and metadata of the images looking at our coordinates
        resp = requests.get(url)
        data = resp.json()

        features = data['features']
        print(len(features))
        for feature in features: 
            image_key = feature['properties']['key']

            # we will use a template link that shows a JPG from Mapillary, and insert the key
            image = 'https://images.mapillary.com/{}/thumb-1024.jpg'.format(image_key)
            
            # This is where you want to store the image
            image_filename = os.path.join(output_image_path,image_key+".jpg")
            
            if os.path.exists(image_filename):
                print('Image already exists')
                continue
            else:
                print('Downloading image', image_filename)

            # request the image URL, to get a displayable image
            r = requests.get(image,stream=all)
            b = BytesIO(r.content)
            img = Image.open(b)
            img.save(image_filename)

            # Also store the JSON, perhaps you might need it for something
            with open(os.path.join(output_json_path, image_key+".json"), 'w') as outfile:
                json.dump(feature, outfile)
    except Exception as e: 
        print(e)
        continue